## T3 - Scenarios

You should now be able to run a single sim, using default or custom parameters (T1) and plot the results (T2). 

In this tutorial, you will learn how to create your own intervention scenarios. The code used to run scenarios is housed in the scenarios.py script, which you can reference as we go. To write custom scenarios, we define a new function - 'Scenario'. 

At the bottom of this tutorial, you can find a list of all the args that Scenario will take. 

Let's start with a basic scenario in which we increase the method efficacy of the existing injectables to 99%. We'll label the scenario s1, to keep it short & sweet. First, call in fpsim using fp. Then, ask it to make a scenario (make_scen), and pass a couple of arguments. First, we want injctables efficacy to increase, and second, we want that change to start in 2020. We can imagine using this kind of scenario when a user wants to <i>improve an existing method</i>, but does not think this will impact behavior. 

In [2]:
import fpsim as fp
s1 = fp.make_scen(eff={'Injectables':0.99}, year = 2020)

If we want to instead change behavior around a method, we can adjust the initiation, switching, and/or discontinuation. In s2, we'll double the initiation rate of injectables. 

In [3]:
s2 = fp.make_scen(method = 'Injectables', init_factor = 2, year = 2020)

init_factor is shorthand for multiplying a specific source-destination dyad by an amount defined by the user (e.g. 0.5 to cut in half, 2.0 to double). So, the equivalent long form of s2 would be: 

In [4]:
s2 = fp.make_scen(source = "None", dest = "Injectables", factor = 2, year = 2020)

Users may also want to specify a specific value, instead of using a factor. In this case, we would input init_value = [your value] instead of a factor. 

In [5]:
s2_value = fp.make_scen(method = 'Injectables', init_value = 0.07, year = 2020) # in this case, we would need robust user insights data to tell us that we should anticipate ~7% uptake

Scenarios can be easily combined, so that you can layer multiple interventions in one scenario. Here, we can boost the efficacy of injectables <i>and</i> double the uptake. 

In [6]:
s3 = s1 + s2

Rarely are we so lucky to have such straightforward interventions, however! Most of the time, the world is much more complex and nuanced. Let's say we think that the newly improved injectables will especially appeal to the 35+ audience when it hits the market in 2027.

In [7]:
s4 = fp.make_scen(method = 'Injectables', init_factor = 2.0, ages = '>35', year = 2027)

We can also combine different changes in a single scenario by using a dictionary. In this case, let's imagine that our newly improved 2027 injectables will appeal to the 35+ audience <i>and</i> will draw not only new users, but current condom users. In this case, we need two lines of changes, one line to affect injectables, and one to affect condom switching behavior. 

In [15]:
s5 = fp.make_scen(
    year = 2027,
    probs = [
        dict(method = 'Injectables', init_factor = 2.0, ages = '>35'),
        dict(source = 'Condoms', dest = 'Injectables', value = 0.20) #assume 20% likelihood of condom users to switch to injectables
    ]
)

If you want to write up several scenarios that share multiple arguments, it's easy to use kwargs to make life easier (and more legible). In this example, we'll take a look at some counterfactual scenarios in which we see big increases in injectable uptake for the under 20 population. We set the pars first as global - so they'll be part of all scenarios. We then set kwargs as a dictionary that specific scenarios will pull. 

In [23]:
    n_agents   = 10_000
    start_year = 1980
    repeats    = 3
    year       = 2012
    youth_ages = ['<18', '18-20']
    
    pars = fp.pars(location = 'senegal', n_agents=n_agents, start_year=start_year, end_year=2020) #set pars to include all shared arguments
    
    method = 'Injectables'
    kw = dict(method=method, ages=youth_ages) #set kwargs to take on the method and the age group
    d_kw = dict(dest=method, ages = youth_ages)
    f1 = fp.make_scen(
            label = '2x uptake',
            year = year,
            probs = [
                dict(init_factor = 2.0, **kw),
                    ]
                )
    f2 = fp.make_scen(
            label = '5x uptake',
            year = year,
            probs = [
                dict(init_factor = 5.0, **kw),
                    ]
                )
    
    f3 = fp.make_scen(
            label = '10x uptake',
            year = year,
            probs = [
                dict(init_factor = 10.0, **kw),
                    ]
                )
    f_switch = fp.make_scen(
            label = '20 percent switching',
            year = year,
            probs = [
                dict(source = 'Injectables', value = 0.20, **d_kw)
                    ]
               )

    f4 = f2 + f_switch # We wil come back to this later.
            


So, now that we've defined the scenarios we want to explore, we need to run them together so that we can compare them. Using the 'f' scenarios from above, we first call on Scenarios to run the sims with the pars and repeats we defined above. We're then going to add in the new scens (including the baseline) one by one. Note: You can label scenarios here and they'll override what you used as a label when defining the scenarios. This can be useful if you're combining two different scenarios and you want to re-label the combination.  

In [20]:

scens = fp.Scenarios(pars=pars, repeats=repeats)
scens.add_scen(label='Baseline')
scens.add_scen(f1)
scens.add_scen(f2)
scens.add_scen(f3)
scens.add_scen(f4, label = '5x uptake plus switching') #combining f2 and f_switch and re-labeling here

In [21]:
#Note: Running the scenarios can take quite a while, depending on how large your starting population is (n_agents) and how many times you'd like to repeat the sim with a new seed (repeats). 
#On a local machine, with n_agents = 10,000 and repeats = 3, this run took the author 3m 2.6s. YMMV!
scens.run()

Congratulations on making it this far! Below you'll find the args for scenarios, which we hope will help you craft your own scenarios. In T4, we'll go through the plotting options for scenarios. 

# Args for Scenarios
 
    Args (shared):
        spec   (dict): a pre-made specification of a scenario; see keyword explanations below (optional)
        args   (list): additional specifications (optional)
        label  (str): the sim label to use for this scenario
        pars   (dict): optionally supply additional sim parameters to use with this scenario (that take effect at the beginning of the sim, not at the point of intervention)
        year   (float): the year at which to activate efficacy and probability scenarios
        matrix (str): which set of probabilities to modify for probability scenarios (e.g. annual or postpartum)
        ages   (str/list): the age groups to modify the probabilities for

    Args (efficacy):
        year (float): as above
        eff  (dict): a dictionary of method names and new efficacy values

    Args (probablity):
        year   (float): as above
        matrix (str): as above
        ages   (str): as above
        source (str): the method to switch from
        dest   (str): the method to switch to
        factor (float): if supplied, multiply the [source, dest] probability by this amount
        value  (float): if supplied, instead of factor, replace the [source, dest] probability by this value
        copy_from (str): if supplied, copy probabilities from a different method

    Args (initiation/discontinuation):
        year   (float): as above
        matrix (str): as above
        ages   (str): as above
        method (str): the method for initiation/discontinuation
        init_factor    (float): as with "factor" above, for initiation (None → method)
        discont_factor (float): as with "factor" above, for discontinuation (method → None)
        init_value     (float): as with "value" above, for initiation (None → method)
        discont_value  (float): as with "value" above, for discontinuation (method → None)

    Args (parameter):
        par (str): the parameter to modify
        par_years (float/list): the year(s) at which to apply the modifications
        par_vals (float/list): the value(s) of the parameter for each year

    Args (custom):
        interventions (Intervention/list): any custom intervention(s) to be applied to the scenario

